<a href="https://colab.research.google.com/github/BuxinSu/ICML_Ranking/blob/main/Notebooks/Ranking_and_Final_Decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Google Drive**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Setting**


In [ ]:
from sklearn.isotonic import isotonic_regression
import cvxpy as cp
import numpy as np
import csv, os
from scipy import stats
from sklearn import metrics
import itertools
import pandas as pd
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import statistics
from scipy.stats import levene
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# **Load CSV file into a pandas DataFrame**

In [ ]:
# Load CSV file into a pandas DataFrame
df = pd.read_csv(r'/content/drive/MyDrive/Research/Ranking_ICML/rank_author_dec.csv')
df = df.drop_duplicates(['submission_id', 'author_id'])

# **Add highest-ranked submission to High_decision list and lowest-ranked to Low_decision list**



In [ ]:

## Organize all the submissions by {author: [submission, rank]}.
author_submission_rank_old = {}
authors = df['author_id'].unique()

for author in authors:
    author_submission_rank_old[author] = []
    submissions = df[df['author_id'] == author]['submission_id'].tolist()
    for i in range(len(submissions)):
        rank = df[(df['submission_id'] == submissions[i]) & (df['author_id'] == author)]['rank'].tolist()[0]
        author_submission_rank_old[author].append((submissions[i], rank))



# Sort submissions by rank.
def sort_submissions(author_submission_rank_old):
    for author in author_submission_rank_old:
        author_submission_rank_old[author].sort(key=lambda x: x[1], reverse=False)
    return author_submission_rank_old

author_submission_rank_old = sort_submissions(author_submission_rank_old)



# If final decesion = Accept (Oral & Poster), record 1; Otherwise, record 0.
df['decision'] = df['decision'].map({'Withdrawn or Desk Rejected': 0.0, 'Reject': 0.0, 'Accept (Poster)': 0.0, 'Accept (Oral & Poster)':1.0})



# Append highest-ranked submission into High_decision list; Append lowest-ranked submission into Low_decision list.
High_decision = []
Low_decision = []
for author in author_submission_rank_old:
        decision = df[df['submission_id'] == author_submission_rank_old[author][0][0]]['decision'].tolist()
        decision = decision[0]
        High_decision.append(decision)

        decision = df[df['submission_id'] == author_submission_rank_old[author][-1][0]]['decision'].tolist()
        decision = decision[0]
        Low_decision.append(decision)


# **Perform Chi Squared Test**


In [ ]:
contingency_table = [[0, 0], [0, 0]]
for i in range(len(High_decision)):
    if High_decision[i] == 1:
        contingency_table[0][0] += 1
    if High_decision[i] == 0:
        contingency_table[0][1] += 1
    if Low_decision[i] == 1:
        contingency_table[1][0] += 1
    if Low_decision[i] == 0:
        contingency_table[1][1] += 1


res = stats.chi2_contingency(np.array(contingency_table), correction=True, lambda_=None)

print("high-ranked submissions:", contingency_table[0][0]/(contingency_table[0][0] + contingency_table[0][1]))
print("low-ranked submissions:", contingency_table[1][0]/(contingency_table[1][0] + contingency_table[1][1]))

print("Chi-square test statistic:", res.statistic)
print("P-value:", res.pvalue)

high-ranked submissions: 0.044709388971684055
low-ranked submissions: 0.022354694485842028
Chi-square test statistic: 9.668654159170737
P-value: 0.0018743857093879365
